In [ ]:
import os
import glob
import numpy as np
import vtk
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from utilities import ReadVTUFile, ThresholdInBetween
from vtk.util.numpy_support import vtk_to_numpy, numpy_to_vtk

In [ ]:
path = "/Users/ana/Documents/AnahitaSeresti/03_MBFValidation/09.InterPatientAnalysis"
patient_metadata = glob.glob(f"{path}/*.xlsx")[0]
VTUFiles = glob.glob(f"{path}/*.vtu")
LabelFiles = glob.glob(f"{path}/*.dat")

In [ ]:
PMD = pd.read_excel(patient_metadata)
print(PMD)
patient_ids = set(PMD['patient-id'])
print(patient_ids)
stenosis_sevirity = PMD['Stenosis Severity']
stenosis_sevirity

In [ ]:
def ReadLabels(InputLabels, TerritoryTag):
    MBF_Labels = {}
    for tag in TerritoryTag:
        MBF_Labels[tag] =  []
    keys = list(MBF_Labels.keys())
    with open(InputLabels, "r") as ifile:
        for i, LINE in enumerate(ifile):
            if i == 0: 
                continue
            line = LINE.split()
            for key in keys:
                if key == "LAD":
                    for k in ['LAD', 'Diag']:
                        if line[1].find(k)>=0: 
                            MBF_Labels[key].append(int(line[0]))
                elif key == "PL":
                    for k in ['PL', 'PDA']:
                        if line[1].find(k)>=0: 
                            MBF_Labels[key].append(int(line[0]))
                else:
                    if line[1].find(key)>=0: 
                        MBF_Labels[key].append(int(line[0]))

    MBF_Labels = {k:v for k,v in MBF_Labels.items() if len(v)>0}
    
    return MBF_Labels

def Normalize(MBF):
    for i in range(MBF.GetPointData().GetNumberOfArrays()):
        arrayname_ = MBF.GetPointData().GetArrayName(i)
        if 'scalars' in arrayname_.lower():
            ArrayName = arrayname_
    ScalarArray = MBF.GetPointData().GetArray(ArrayName)
    per_75th = np.percentile(vtk_to_numpy(ScalarArray), 75)
    IndexMBFArray = ScalarArray/per_75th
    IndexMBF = numpy_to_vtk(IndexMBFArray)
    IndexMBF.SetName("IndexMBF")
    MBF.GetPointData().AddArray(IndexMBF)

    return MBF

def CollectMBFData(MBF, Labels):
    for i in range(MBF.GetPointData().GetNumberOfArrays()):
            arrayname_ = MBF.GetPointData().GetArrayName(i)
            if 'scalars' in arrayname_.lower():
                ScalarArray = arrayname_

    AbsMBFData = {}
    IndexMBFData = {}
    for key in Labels.keys():
        AbsMBFData[key] = np.array([])
        IndexMBFData[key] = np.array([])
        for i in Labels[key]:
            territory_ = ThresholdInBetween(MBF, "TerritoryMaps", i, i)
            MBF_ = vtk_to_numpy(territory_.GetPointData().GetArray(ScalarArray))
            iMBF_ = vtk_to_numpy(territory_.GetPointData().GetArray("IndexMBF"))
            AbsMBFData[key] = np.append(AbsMBFData[key], MBF_)
            IndexMBFData[key] = np.append(IndexMBFData[key], iMBF_)
    

    return AbsMBFData, IndexMBFData

In [ ]:
data = []
for id in patient_ids:
    print(id)
    tags = list(PMD[PMD['patient-id'] == id]['stenosis tag'])
    is_ischemic = list(PMD[PMD['patient-id'] == id]['is-ischemic'])
    FFR = list(PMD[PMD['patient-id'] == id]['FFR class'])
    FFRValue = list(PMD[PMD['patient-id'] == id]['FFR-Value'])
    severity = list(PMD[PMD['patient-id'] == id]['Stenosis Severity'])
    for file in LabelFiles:
        if id in os.path.basename(os.path.splitext(file)[0]):
            Labels = file
    MBFLabels = ReadLabels(Labels, tags)
    for file in VTUFiles:
        if id in os.path.basename(os.path.splitext(file)[0]):
            MBF = Normalize(ReadVTUFile(file))
    AbsMBFData, IndexMBFData = CollectMBFData(MBF, MBFLabels)
    for i, t in enumerate(tags):
        data_ = {"patient_id": id, "tag": t}
        
        if is_ischemic[i] == 1:
            data_["is_ischemic"] = True
        else:
            data_["is_ischemic"] = False
        if FFR[i] == 1:
            data_["FFR"] = True
        elif FFR[i] == 0:
            data_["FFR"] = False
        else:
            data_["FFR"] = 'na'
        
        data_["Severity"] = severity[i]
        data_["FFRValue"] = FFRValue[i]
        data_["absMBF"] = AbsMBFData[t]
        indexMBF = IndexMBFData[t]
        data_["IndexMBF"] =  indexMBF
        data_["MeanIndexMBF"] = np.mean(indexMBF[indexMBF != 0.0])
        data.append(data_)

df = pd.DataFrame(data)
df



In [ ]:
data = []
for id in patient_ids:
    print(id)
    tags = list(PMD[PMD['patient-id'] == id]['stenosis tag'])
    is_ischemic = list(PMD[PMD['patient-id'] == id]['is-ischemic'])
    FFR = list(PMD[PMD['patient-id'] == id]['FFR class'])
    for file in LabelFiles:
        if id in os.path.basename(os.path.splitext(file)[0]):
            Labels = file
    MBFLabels = ReadLabels(Labels, tags)
    for file in VTUFiles:
        if id in os.path.basename(os.path.splitext(file)[0]):
            MBF = Normalize(ReadVTUFile(file))
    AbsMBFData, IndexMBFData = CollectMBFData(MBF, MBFLabels)
    for i, t in enumerate(tags):
        data_ = {"patient_id": id, "tag": t, "is_ischemic": is_ischemic[i], "FFR": FFR[i], "absMBF": AbsMBFData[t], "IndexMBF": IndexMBFData[t]}
        data.append(data_)

df2 = pd.DataFrame(data)
df2

In [ ]:
index_mbf_ischemic = np.concatenate(df[df['is_ischemic'] == True]['IndexMBF'].values)
q1, q2, q3 = np.percentile(index_mbf_ischemic, [25,50,75])
#print(q1, q2, q3)
index_mbf_nonischemic = np.concatenate(df[df['is_ischemic'] == False]['IndexMBF'].values)
q1, q2, q3 = np.percentile(index_mbf_nonischemic, [25,50,75])
#print(q1, q2, q3)

index_mbf_FFR1 = np.concatenate(df[df["FFR"] == True]["IndexMBF"].values)
q1, q2, q3 = np.percentile(index_mbf_FFR1, [25,50,75])
print(q2, q3-q1)

index_mbf_FFR0 = np.concatenate(df[(df["FFR"] == False) & (df["is_ischemic"] == False)]["IndexMBF"].values)
q1, q2, q3 = np.percentile(index_mbf_FFR0, [25,50,75])
print(q2, q3-q1)

index_mbf_severity0 = np.concatenate(df[df['Severity'] == 0]['IndexMBF'].values)
q1, q2, q3 = np.percentile(index_mbf_severity0, [25,50,75])
print(q2, q3-q1)
index_mbf_severity1 = np.concatenate(df[df['Severity'] == 1]['IndexMBF'].values)
q1, q2, q3 = np.percentile(index_mbf_severity1, [25,50,75])
print(q2, q3-q1)
index_mbf_severity3 = np.concatenate(df[df['Severity'] == 3]['IndexMBF'].values)
q1, q2, q3 = np.percentile(index_mbf_severity3, [25,50,75])
print(q2, q3-q1)
index_mbf_severity4 = np.concatenate(df[df['Severity'] == 4]['IndexMBF'].values)
q1, q2, q3 = np.percentile(index_mbf_severity4, [25,50,75])
print(q2, q3-q1)




In [ ]:
df_plot = pd.DataFrame({
    'IndexMBF': np.concatenate([index_mbf_ischemic, index_mbf_nonischemic, index_mbf_FFR1, index_mbf_FFR0]),
    'Group': ['Ischemic Territories']*len(index_mbf_ischemic) +
            ['Nonischemic Territories']*len(index_mbf_nonischemic) +
            ['Flow-limiting Stnosis']*len(index_mbf_FFR1) +
            ['Insignificant Stenosis']*len(index_mbf_FFR0), 
    'Type': ['Perfusion']*(len(index_mbf_ischemic )+ len(index_mbf_nonischemic)) + ['FFR']*(len(index_mbf_FFR1) + len(index_mbf_FFR0))
})



In [ ]:
df_plot = pd.DataFrame({
    'IndexMBF': np.concatenate([index_mbf_severity0, index_mbf_severity1, index_mbf_severity3, index_mbf_severity4, index_mbf_FFR1, index_mbf_FFR0]),
    'Group': ['No\nStenosis']*len(index_mbf_severity0) +
            ['Mild\nStenosis']*len(index_mbf_severity1) +
            ['Moderate\nStenosis']*len(index_mbf_severity3) +
            ['Severe\nStenosis']*len(index_mbf_severity4) +
            ['Flow-\nlimiting\nStenosis']*len(index_mbf_FFR1) +
            ['Insignificant\nStenosis']*len(index_mbf_FFR0), 
    'Type': ['CCTA']*(len(index_mbf_severity0 )+ len(index_mbf_severity1)+ len(index_mbf_severity3 )+ len(index_mbf_severity4)) + ['FFR']*(len(index_mbf_FFR1) + len(index_mbf_FFR0))
})



In [ ]:
plt.figure(figsize=(5, 4))

ax = sns.boxplot(x='Group', y='IndexMBF', hue='Type', width=0.2,
                data= df_plot, showfliers=False, palette={'CCTA':'salmon', 'FFR':'mediumspringgreen'})

ax.set_ylabel("Index MBF", fontsize = 10)
plt.setp(ax.get_xticklabels(), rotation=0, ha='center', fontsize = 8)
plt.legend(loc = 'upper right', fontsize = 8)
plt.tight_layout()
plt.show()

In [ ]:
groups = df_plot['Group'].unique()
types = df_plot['Type'].unique()
data = []
labels = []
colors = []
color_map = {'Anatomical Severity': 'salmon', 'FFR': 'mediumspringgreen'}
for group in groups:
    for t in types:
        subset = df_plot[(df_plot['Group'] == group) & (df_plot['Type'] == t)]
        data.append(subset['IndexMBF'].values)
        labels.append(f'{group}')
        colors.append(color_map[t])

print(labels)


In [ ]:
print(groups)

In [ ]:

_, ax = plt.subplots(figsize=(8, 5))
box = ax.boxplot(data, patch_artist=True, showfliers=False)

for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)

ax.set_ylabel("Index MBF", fontsize=20)
ax.set_xticks(range(1, len(groups) + 1))
ax.set_xticklabels(groups, rotation=60, ha='right', fontsize=15)

for median in box['medians']:
    median.set_color('darkblue')
    median.set_linewidth(2)

handles = [plt.Line2D([0], [0], color=color, lw=10) for color in color_map.values()]
ax.legend(handles, color_map.keys(), loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
#average_index_mbf = np.array([np.mean(x) for x in df[df["FFR"] != 'na']['IndexMBF'].values])
average_index_mbf = np.array([x for x in df[df["FFR"] != 'na']['MeanIndexMBF']])

In [ ]:
FFRvalues = np.array([x for x in df[df["FFR"] != 'na']['FFRValue']])
FFRvalues

In [ ]:
from scipy.stats import pearsonr, linregress

print(pearsonr(FFRvalues, average_index_mbf))

In [ ]:
plt.scatter(FFRvalues, average_index_mbf, color = 'firebrick')

slope, intercept, _, _, _ = linregress(FFRvalues, average_index_mbf)
x_fit = np.linspace(min(FFRvalues), max(FFRvalues), 100)
y_fit = slope * x_fit + intercept

plt.plot(x_fit, y_fit, 'k--', label=f"Trend line (r={pearsonr(FFRvalues, average_index_mbf)[0]:.2f})")
plt.plot(x_fit, x_fit, 'k-', label="x = y line")

plt.xlabel("FFR", fontsize = 12)
plt.ylabel("Mean Index MBF", fontsize = 12)
plt.legend(fontsize = 8)

plt.show()
